<a href="https://colab.research.google.com/github/Amnariaz640/Pytorch-/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [3]:
train_data = torchvision.datasets.CIFAR10(root='/Classification_data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='/Classification_data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
image, label = train_data[0]

In [5]:
image.size()

torch.Size([3, 32, 32])

In [6]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [25]:
class NeuralNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, 5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(16, 32, 5)
    self.fc1 = nn.Linear(32*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84,10)

  def forward(self, x):
     x = F.relu(F.max_pool2d(self.conv1(x), 2))
     x = F.relu(F.max_pool2d(self.conv2(x), 2))
     x = x.view(-1, self.num_flat_features(x))  # Flatten the tensor
     x = F.relu(self.fc1(x))
     x = F.relu(self.fc2(x))
     x = self.fc3(x)
     return x

  def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [27]:
net = NeuralNet()
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr =0.01, momentum = 0.9)

In [28]:
for epoch in range(30):
    print(f'Training Epoch {epoch + 1}')
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get the inputs and labels from the dataloader
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)

        # Compute the loss
        loss_value = loss(outputs, labels)  # Use `loss` here to compute the loss

        # Backward pass and optimize
        loss_value.backward()
        optimizer.step()

        # Accumulate the running loss
        running_loss += loss_value.item()

    # Print average loss for the epoch
    print(f' Loss: {running_loss / len(train_loader):.4f}')

Training Epoch 1
 Loss: 1.6320
Training Epoch 2
 Loss: 1.1942
Training Epoch 3
 Loss: 1.0125
Training Epoch 4
 Loss: 0.9045
Training Epoch 5
 Loss: 0.8190
Training Epoch 6
 Loss: 0.7557
Training Epoch 7
 Loss: 0.7013
Training Epoch 8
 Loss: 0.6578
Training Epoch 9
 Loss: 0.6187
Training Epoch 10
 Loss: 0.5746
Training Epoch 11
 Loss: 0.5508
Training Epoch 12
 Loss: 0.5317
Training Epoch 13
 Loss: 0.4967
Training Epoch 14
 Loss: 0.4854
Training Epoch 15
 Loss: 0.4755
Training Epoch 16
 Loss: 0.4534
Training Epoch 17
 Loss: 0.4438
Training Epoch 18
 Loss: 0.4363
Training Epoch 19
 Loss: 0.4152
Training Epoch 20
 Loss: 0.4102
Training Epoch 21
 Loss: 0.4100
Training Epoch 22
 Loss: 0.4167
Training Epoch 23
 Loss: 0.4080
Training Epoch 24
 Loss: 0.4148
Training Epoch 25
 Loss: 0.3760
Training Epoch 26
 Loss: 0.4031
Training Epoch 27
 Loss: 0.4070
Training Epoch 28
 Loss: 0.4165
Training Epoch 29
 Loss: 0.4018
Training Epoch 30
 Loss: 0.3972


In [29]:
torch.save(net.state_dict(), 'trained.pth')

In [30]:
net = NeuralNet()
net.load_state_dict (torch.load('trained.pth'))

<ipython-input-30-4f777d5cbf82>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict (torch.load('trained.pth'))


<All keys matched successfully>

In [31]:
correct = 0
total  = 0
net.eval()

with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct+= (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 65.85%


In [49]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0) # Correct the typo here
    return image


image_paths = ['pic5.jfif']
images = [load_image(img) for img in image_paths]

net = NeuralNet()
net.load_state_dict(torch.load('trained.pth', map_location=torch.device('cpu'), weights_only=True))  # Load weights only
net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted = torch.max(output.data, 1)
    print(f'Prediction: {class_names[predicted.item()]}')


Prediction: bird
